# Space

In [ ]:
import sys
import os 
import logging
import pandas as pd
from pprint import pprint 
from IPython.display import display, HTML

KEY = '1-WORKSPACE'
WORKSPACE_PATH = os.getcwd().split(KEY)[0]
print(WORKSPACE_PATH); os.chdir(WORKSPACE_PATH)
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='[%(levelname)s:%(asctime)s:(%(filename)s@%(lineno)d %(name)s)]: %(message)s')

SPACE = {
    'DATA_RAW': f'./_Data/0-Data_Raw',
    'DATA_RFT': f'./_Data/1-Data_RFT',
    'DATA_CASE': f'./_Data/2-Data_CASE',
    'DATA_AIDATA': f'./_Data/3-Data_AIDATA',
    'DATA_EXTERNAL': f'./code/external',
    'CODE_FN': f'./code/pipeline', 
}
assert os.path.exists(SPACE['CODE_FN']), f'{SPACE["CODE_FN"]} not found'

print(SPACE['CODE_FN'])
sys.path.append(SPACE['CODE_FN'])

# Step 1: Cohort and Human

In [ ]:
from config.config_record.Cohort import CohortName_to_OneCohortArgs

# %%%%%%%%%%%%%%%%%%%%% user
CohortName = 'WellDoc2023CVSDeRx'
# CohortName = 'OhioT1DM'
# CohortName = 'aireadi-noimage-v2'
# CohortName = 'WellDoc2025ALS'
# CohortName = 'WellDoc2025LLY'
OneCohort_Args = CohortName_to_OneCohortArgs[CohortName]
# %%%%%%%%%%%%%%%%%%%%%


In [ ]:
# %%%%%%%%%%%%%%%%%%%%% user
HumanName = 'P'
# %%%%%%%%%%%%%%%%%%%%% user


In [ ]:
from recfldtkn.record_base.cohort import CohortFn, Cohort
Source2CohortName = OneCohort_Args['Source2CohortName']
cohort_fn = CohortFn(Source2CohortName, SPACE)
cohort = Cohort(OneCohort_Args, SPACE, cohort_fn)
cohort.setup_fn(cohort_fn)
cohort.initialize_cohort()

In [ ]:
cohort.pypath

In [ ]:
cohort.RawName_to_dfRaw

In [ ]:
from recfldtkn.record_base.human import HumanFn, Human   

human_fn = HumanFn(HumanName, SPACE)
human = Human(HumanName, cohort, human_fn)
human.setup_fn(human_fn)
human.initialize_human()

In [ ]:
human.pypath

In [ ]:
human.df_Human

# Step 2: OneRecord_Args

In [ ]:
# %%%%%%%%%%%%%%%%%%%%% user
RecordName = 'CGM5Min'
# %%%%%%%%%%%%%%%%%%%%% user

## %%%%%%%%%%%%%%%%%%%%% user
OneRecord_Args = {
    'RecordName': 'CGM5Min', 
    'RecID': 'CGM5MinID',
    'RecIDChain': ['PID'],

    'RawHumanID': 'PatientID',
    'ParentRecName': 'P', 

    'RecDT': 'DT_s', 
    'RawNameList': ['ElogBGEntry'],
    'human_group_size': 100,
    'rec_chunk_size': 100000,
}
## %%%%%%%%%%%%%%%%%%%%% user

In [ ]:
RawName_to_dfRaw = cohort.RawName_to_dfRaw
[RawName for RawName in RawName_to_dfRaw]

In [ ]:
from recfldtkn.record_base.record import RecordFn, Record

record_fn = None 
record = Record(RecordName, human, record_fn)
record 

In [ ]:
OneCohort_Args = cohort.OneCohort_Args
RawName_to_dfRaw = cohort.RawName_to_dfRaw
df_Human = human.df_Human

# Step 3: RecordPrtInfo

In [ ]:
OneHuman_Args = human.OneHuman_Args

In [ ]:
OneCohort_Args

In [ ]:
if OneRecord_Args['ParentRecName'] is not None:

    # full version
    record_prt = Record(OneRecord_Args['ParentRecName'], human)
    record_prt.setup_fn()
    record_prt.setup_prt()
    record_prt.initialize_record()

    # neat version
    # record_prt = Record(OneRecord_Args['ParentRecName'], human)
    # record_prt.setup_fn()
    # record_prt.initialize_record()
else:
    record_prt = None 
    
print('record_prt:', record_prt)

In [ ]:
human = human
RecordPrtInfo = record.get_RecordPrtInfo(OneRecord_Args, 
                                         human, 
                                         record_prt) 
df_Human = human.df_Human
RecordPrtInfo
# 

In [ ]:
df_Human.columns

# Step 4: Display Record's RawColumns

In [ ]:
OneRecord_Args['RawNameList']

In [ ]:
RawNameList = OneRecord_Args['RawNameList']
RawName_to_dfRaw = cohort.RawName_to_dfRaw
RawName_to_Sample = record.display_Record_RawNameCols(RawNameList, RawName_to_dfRaw)

In [ ]:
for RawName, df_sample in RawName_to_Sample.items():
    print(f'\n===== {RawName} =====')
    display(df_sample)

In [ ]:
# df_sample['EntrySourceID'].value_counts()

# Step 5: Select Useful Raw Columns


In [ ]:
OneRecord_Args['RawNameList']

In [ ]:
## %%%%%%%%%%%%%%%%%%%%% user
RawName_to_RawConfig = {}

RawName = 'ElogBGEntry'
raw_columns = [
 'BGEntryID', 
 'PatientID',
 # 'EntryID', 
 'ObservationDateTime', 
 'BGValue',
#  'IsNormalIndicator', 
 # 'ParentEntryID', 'ActivityTypeID',
 'ObservationEntryDateTime', 
 'TimezoneOffset', 
#  'Timezone', # 'BGRecheckStartTime',
 # 'BGRecheckDuration', 'EntrySourceID', 
 'EntryCreatedDateTime',
 # 'ObservationCreatedBy', 'ObservationStatus', 'RowVersionID', 'MeterType',
 # 'IsFeedbackShown', 'SourceReferenceID', 'ModifiedDateTime', 'ObservationUOMID',
 # 'ExternalEntryID', 'BGCode', 
 'ActualBGValue', # 
 'ExternalSourceID',
 'UserObservationDateTime'
 ]

raw_datetime_column = 'ObservationDateTime' 

raw_base_columns = ['BGEntryID', 
 'PatientID',
 # 'EntryID', 
 'ObservationDateTime', 
 # 'ParentEntryID', 'ActivityTypeID',
 'ObservationEntryDateTime', 
 'TimezoneOffset', 
#  'Timezone', # 'BGRecheckStartTime',
 # 'BGRecheckDuration', 'EntrySourceID', 
 'EntryCreatedDateTime',
 # 'ObservationCreatedBy', 'ObservationStatus', 'RowVersionID', 'MeterType',
 # 'IsFeedbackShown', 'SourceReferenceID', 'ModifiedDateTime', 'ObservationUOMID',
 # 'ExternalEntryID', 'BGCode', 
 # 
 'ExternalSourceID',
 'UserObservationDateTime'
 ]


RawConfig = {
    'raw_columns': raw_columns, 
    'raw_base_columns': raw_base_columns,
    'rec_chunk_size': 100000,
    'raw_datetime_column': raw_datetime_column,
}
RawName_to_RawConfig[RawName] = RawConfig
## %%%%%%%%%%%%%%%%%%%%% user

RawName_to_RawConfig

# Step 6: Get df_HumanSelected

In [ ]:
[RawName for RawName in OneRecord_Args['RawNameList']]

In [ ]:
RawName_to_dfRaw

In [ ]:
OneCohort_Args = cohort.OneCohort_Args
OneHuman_Args = human.OneHuman_Args
OneRecord_Args = OneRecord_Args
df_Human = human.df_Human

df_HumanSelected = record.get_dfHumanSelected(OneCohort_Args, OneHuman_Args, OneRecord_Args, df_Human)
df_HumanSelected

# Step 7: Get df_HumanGroup

In [ ]:
for human_group, df_HumanGroup in df_HumanSelected.groupby('human_group'):
    logger.info(f'\n====={human_group}=====')
    display(df_HumanGroup)
    break

# Step 8: Get df_RawRec_for_HumanGroup

In [ ]:
df_RawRec_for_HumanGroup = record.get_dfRawRec_from_dfHumanGroup(OneHuman_Args,
                                                                df_HumanGroup,
                                                                RawName_to_RawConfig, 
                                                                RawName_to_dfRaw)
df_RawRec_for_HumanGroup

# Step 9: Get df_RawRecProc_for_HumanGroup

In [ ]:
df_RawRec_for_HumanGroup['ObservationDateTime'] = pd.to_datetime(df_RawRec_for_HumanGroup['ObservationDateTime']).dt.tz_localize(None)



df_RawRec_for_HumanGroup['BGValue'] = pd.to_numeric(df_RawRec_for_HumanGroup['BGValue'], errors='coerce')
df_RawRec_for_HumanGroup = df_RawRec_for_HumanGroup[df_RawRec_for_HumanGroup['BGValue'].notna()].reset_index(drop = True)

print(df_RawRec_for_HumanGroup.shape)


In [ ]:
# df = df_RawRec_for_HumanGroup.copy()
# # df = df[df['TimezoneOffset'].abs() < 1000].reset_index(drop = True)
# df

In [ ]:
# DTCol_list = [
#     'ObservationDateTime', 
#     # 'ParentEntryID', 'ActivityTypeID',
#     'ObservationEntryDateTime', 
#     'EntryCreatedDateTime',
#     'UserObservationDateTime'
#     ]

# for DTCol in DTCol_list: 
#     df[DTCol] = pd.to_datetime(df[DTCol], format = 'mixed')

In [ ]:
# # x1. localize the datetime columns to based on time zone. 
# a = len(df)
# df = pd.merge(df, df_Human[['PatientID', 'user_tz']],  how = 'left')
# b = len(df)
# assert a == b

# df

In [ ]:
import inspect 

## %%%%%%%%%%%%%%%%%%%%% user
def get_RawRecProc_for_HumanGroup(df_RawRec_for_HumanGroup, OneRecord_Args, df_Human):
    df = df_RawRec_for_HumanGroup
    
    # 1. filter out the records we don't need (optional) 

    ##########
    # index = (df['ExternalSourceID'] == 18) # | (df['MeterType'] == 5)  # CGM
    # df = df[index].reset_index(drop = True)
    ##########



    df = df[df['TimezoneOffset'].abs() < 1000].reset_index(drop = True)

    # 2. create a new column for raw record id (optional)

    # 3. update datetime columns 
    DTCol_list = [
        'ObservationDateTime', 
        # 'ParentEntryID', 'ActivityTypeID',
        # 'ObservationEntryDateTime', 
        # 'EntryCreatedDateTime',
        # 'UserObservationDateTime'
        ]

    for DTCol in DTCol_list: 
        df[DTCol] = pd.to_datetime(df[DTCol], format = 'mixed')

    # x1. localize the datetime columns to based on time zone. 
    a = len(df)
    df = pd.merge(df, df_Human[['PatientID', 'user_tz']],  how = 'left')
    b = len(df)
    assert a == b

    # Ensure 'TimezoneOffset' is of the correct type and handle missing values
    # df['DT_tz'] = (
    #     df['TimezoneOffset']
    #     .infer_objects(copy=False)
    #     .replace(0, pd.NA)  # Use pd.NA for missing values
    #     .fillna(df['user_tz'])  # Fill missing values with 'user_tz'
    #     .astype('Int64')  # Ensure the column is of integer type, if applicable
    # )

    # df['DT_tz'] = (
    #     df['TimezoneOffset']
    #     .infer_objects(copy=False)
    #     .replace(0, pd.NA)  # Use pd.NA for missing values
    #     .fillna(df['user_tz'])  # Fill missing values with 'user_tz'
    #     .infer_objects(copy=False)  # Infer the data type after fillna
    #     .astype('Int64')  # Ensure the column is of integer type, if applicable
    # )

    df['DT_tz'] = pd.Series(
        df['TimezoneOffset'].replace(0, pd.NA).to_numpy(),
        dtype=pd.Int64Dtype()
    ).fillna(df['user_tz'])



    # print(df['DT_tz'])

    # If 'DT_tz' is supposed to be a datetime column, convert it
    # df['DT_tz'] = pd.to_datetime(df['DT_tz'], errors='coerce')
    # df['DT_tz'] = df['TimezoneOffset'].infer_objects(copy=False).replace(0, None).fillna(df['user_tz'])


    # DTCol = 'DT_r'
    # DTCol_source = 'EntryCreatedDateTime'
    # df[DTCol] = df[DTCol_source]
    # df[DTCol] = pd.to_datetime(df[DTCol]) + pd.to_timedelta(df['DT_tz'], 'm')
    # assert df[DTCol].isna().sum() == 0


    DTCol = 'DT_s'
    DTCol_source = 'ObservationDateTime'
    df[DTCol] = df[DTCol_source]
    df[DTCol] = pd.to_datetime(df[DTCol]).apply(lambda x: None if x <= pd.to_datetime('2010-01-01') else x)
    

    df[DTCol] = pd.to_datetime(df[DTCol]) + pd.to_timedelta(df['DT_tz'], 'm')


    # df[DTCol] = df[DTCol].fillna(df['DT_r'])

    
    df = df[df[DTCol].notna()].reset_index(drop = True)
    # assert df[DTCol].isna().sum() == 0

    DTCol_list = ['DT_s', 
                  # 'DT_r'
                  ] # 'DT_e'
    for DTCol in DTCol_list:
        # DateTimeUnit ='5Min'
        date = df[DTCol].dt.date.astype(str)
        hour = df[DTCol].dt.hour.astype(str)
        minutes = ((df[DTCol].dt.minute / 5).astype(int) * 5).astype(str)
        df[DTCol] = pd.to_datetime(date + ' ' + hour +':' + minutes + ':' + '00')

    # x3. drop duplicates
    df = df.drop_duplicates()

    # 4. select a DT as the RecDT
    RecDT = 'DT_s'

    # ----------------------------------------------------------------- #
    # x4. get the BGValue mean by RecDT (5Min)
    RawHumanID = OneRecord_Args['RawHumanID']
    df = df.groupby([RawHumanID, RecDT])[['BGValue']].mean().reset_index()
    # ----------------------------------------------------------------- #
    
    df_RawRecProc = df
    return df_RawRecProc 
## %%%%%%%%%%%%%%%%%%%%% user

get_RawRecProc_for_HumanGroup.fn_string = inspect.getsource(get_RawRecProc_for_HumanGroup)

In [ ]:
# df_RawRec_for_HumanGroup['BGValue'].astype(float)

df_RawRec_for_HumanGroup.sort_values(by = ['PatientID', 'ObservationDateTime'])

In [ ]:
df_RawRecProc_for_HumanGroup = get_RawRecProc_for_HumanGroup(df_RawRec_for_HumanGroup, OneRecord_Args, df_Human)
df_RawRecProc_for_HumanGroup 

# Step 10: Get dfRecAttr from dfRawRec

In [ ]:
df_RecAttr_for_HumanGroup = record.get_dfRecAttr_from_dfRawRec(df_RawRecProc_for_HumanGroup, OneRecord_Args, RecordPrtInfo)
df_RecAttr_for_HumanGroup

# Step 11: Attr_Cols

In [ ]:
## %%%%%%%%%%%%%%%%%%%%% user
attr_cols = [
    'PID', 'PatientID', 'CGM5MinID',
    'DT_s', 'BGValue', 
]
## %%%%%%%%%%%%%%%%%%%%% user

In [ ]:
df_RecAttr_for_HumanGroup = df_RecAttr_for_HumanGroup[attr_cols].reset_index(drop=True)
df_RecAttr_for_HumanGroup

# Step 12: ds_RecAttr

In [ ]:
ds_RecAttr = record.get_dsRecAttr(OneRecord_Args = OneRecord_Args, 
                                  human = human, 
                                  RawName_to_RawConfig = RawName_to_RawConfig,
                                  attr_cols = attr_cols,
                                  get_RawRecProc_for_HumanGroup = get_RawRecProc_for_HumanGroup,
                                  record_prt = record_prt, 
                                  RecordPrtInfo = RecordPrtInfo)
ds_RecAttr

# Step 13: ds_RecIndex

In [ ]:
ds_RecIndex = record.get_dsRecIndex(OneHuman_Args, OneRecord_Args, ds_RecAttr)
ds_RecIndex

In [ ]:
ds_RecIndex.to_pandas()

# Step 14: Save Fn

In [ ]:
pypath = record.pypath
print(pypath)

from recfldtkn.base import Base 
prefix = ['import pandas as pd', 'import numpy as np']
iterative_variables = [OneRecord_Args, RawName_to_RawConfig, attr_cols]
fn_variables = [get_RawRecProc_for_HumanGroup]
pycode = Base.convert_variables_to_pystirng(fn_variables = fn_variables, 
                                            iterative_variables = iterative_variables,
                                            prefix = prefix)
if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
with open(pypath, 'w') as file: file.write(pycode)

# Step 15: Test

In [ ]:
from recfldtkn.record_base.cohort import CohortFn, Cohort

from config.config_record.Cohort import CohortName_to_OneCohortArgs

# %%%%%%%%%%%%%%%%%%%%% user
CohortName = 'WellDoc2023CVSDeRx'
OneCohort_Args = CohortName_to_OneCohortArgs[CohortName]
# %%%%%%%%%%%%%%%%%%%%%

Source2CohortName = OneCohort_Args['Source2CohortName']
cohort_fn = CohortFn(Source2CohortName, SPACE)
cohort = Cohort(OneCohort_Args, SPACE, cohort_fn)
cohort.setup_fn(cohort_fn)
cohort.initialize_cohort()

In [ ]:
from recfldtkn.record_base.human import HumanFn, Human   

# %%%%%%%%%%%%%%%%%%%%% user
HumanName = 'P'
# %%%%%%%%%%%%%%%%%%%%% user

human_fn = HumanFn(HumanName, SPACE)
human = Human(HumanName, cohort, human_fn)
human.setup_fn(human_fn)
human.initialize_human()

In [ ]:
from recfldtkn.record_base.record import RecordFn, Record
# %%%%%%%%%%%%%%%%%%%%%
RecordName = 'CGM5Min' # Pat
# %%%%%%%%%%%%%%%%%%%%%

record = Record(RecordName, human)
record.setup_fn()
record.setup_prt() # <--- you can also put a given record_prt here.
record.initialize_record()    

In [ ]:
record.ds_RecAttr

In [ ]:
record.ds_RecIndex